In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [3]:
base_dir = "data/"

In [4]:
# kurupical
df1 = pd.read_csv(f"{base_dir}/model275/epoch11.csv")
# df2 = pd.read_csv(f"{base_dir}/model276/epoch10.csv")
df3 = pd.read_csv(f"{base_dir}/model277/epoch11.csv")
df4 = pd.read_csv(f"{base_dir}/model277_2/epoch11.csv")
df5 = pd.read_csv(f"{base_dir}/model288_3/epoch19.csv")
df6 = pd.read_csv(f"{base_dir}/model294/epoch18.csv")
df7 = pd.read_csv(f"{base_dir}/model288_4/epoch24.csv")

In [5]:
# takoi-san
df_val_row = pd.read_feather("../input/riiid-test-answer-prediction/train_transformer_last2500k_only_row_id.feather")

def extract_val_row(path):
    df = pd.read_csv(path)
    return pd.merge(df, df_val_row, how="inner")

df101 = extract_val_row(f"{base_dir}/ex67_68_70_74_75_predict.csv/ex67_68_70_74_75_predict.csv")
df102 = extract_val_row(f"{base_dir}/lgb014.csv/lgb014.csv") 
df103 = extract_val_row(f"{base_dir}/ex82_predict.csv/ex82_predict.csv")
df104 = extract_val_row(f"{base_dir}/ex85_predict.csv/ex85_predict.csv")
df105 = extract_val_row(f"{base_dir}/ex86_predict.csv/ex86_predict.csv")

In [6]:
df5.columns, df6.columns

(Index(['Unnamed: 0', 'predict', 'target'], dtype='object'),
 Index(['Unnamed: 0', 'predict', 'target'], dtype='object'))

In [7]:
def sigmoid(x):
    return 1 / (1 + np.e**-x)

In [8]:
# merge
df = pd.DataFrame()
df["target"] = df1["target"]
df["model275"] = df1["predict"]
# df["model276"] = df2["predict"]
# df["model277"] = df3["predict"]
df["model277_2"] = df4["predict"]
df["model288_3"] = df5["predict"]
df["model294"] = df6["predict"]
df["model288_4"] = df7["predict"]
# df["ex68"] = sigmoid(df101["ex68_transformer_75m"])
# df["ex70"] = sigmoid(df101["ex70_transformer_75m"])
df["ex67"] = sigmoid(df101["ex67_lstm_75m"])
df["ex74"] = sigmoid(df101["ex74_transformer_conv1d_75m"])
# df["ex75"] = sigmoid(df101["ex75_transformer_linear_75m"])
df["lgb014"] = df102["ex014_lgb"]
df["ex82"] = sigmoid(df103["ex82_transformer_75m"])
df["ex85"] = sigmoid(df104["ex85_transformer_75m"])
df["ex86"] = sigmoid(df105["ex86_lstm_75m"])

# df["ex55"] = df4["ex55"]
# df["ex56"] = df4["ex56"]
# df["ex57"] = df4["ex57"]

In [9]:
from scipy.optimize import minimize

In [10]:
X = df.drop("target", axis=1).values
y = df["target"].values

In [11]:
def auc(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    weights = weights / weights.sum()
    auc = roc_auc_score(y, (X*weights).sum(axis=1))
    return -auc

In [12]:
initial_weight = np.ones(len(df.columns)-1)
coef_ = minimize(auc, initial_weight, method='nelder-mead')

In [13]:
coef_

 final_simplex: (array([[ 0.40751513,  0.19931971,  3.14111179,  1.32114717,  0.27590745,
         0.023674  ,  1.37974106,  1.43674849, -0.12025644,  4.51680562,
         0.84483714],
       [ 0.40750225,  0.19931526,  3.14109937,  1.3211441 ,  0.2759275 ,
         0.02365666,  1.37975163,  1.43676569, -0.12027483,  4.5167925 ,
         0.84489423],
       [ 0.40750822,  0.19930552,  3.14113781,  1.32116852,  0.27589793,
         0.02364619,  1.37975719,  1.43676134, -0.1202583 ,  4.51680665,
         0.84488603],
       [ 0.40751218,  0.1993117 ,  3.14110071,  1.32112077,  0.27591902,
         0.02367017,  1.37974997,  1.4367577 , -0.12028355,  4.51679794,
         0.84487419],
       [ 0.40750698,  0.19932606,  3.14110966,  1.32115665,  0.275905  ,
         0.02364311,  1.3797602 ,  1.43677848, -0.12026205,  4.51680899,
         0.84487962],
       [ 0.40750764,  0.19930913,  3.14114951,  1.32116214,  0.27587548,
         0.02365849,  1.37975009,  1.43677514, -0.1202601 ,  4.5167841

In [14]:
weight_norm = coef_.x / coef_.x.sum()

In [15]:
weight_norm

array([ 0.03035144,  0.01484519,  0.23394778,  0.0983981 ,  0.02054939,
        0.00176322,  0.10276214,  0.10700801, -0.00895661,  0.33640848,
        0.06292287])

In [16]:
roc_auc_score(y[-500000:], (X[-500000:]*coef_.x).sum(axis=1))

0.8133208580085063

In [17]:
for i in range(len(weight_norm)):
    print("{}: {}".format(df.columns[i+1], round(weight_norm[i], 4)))

model275: 0.0304
model277_2: 0.0148
model288_3: 0.2339
model294: 0.0984
model288_4: 0.0205
ex67: 0.0018
ex74: 0.1028
lgb014: 0.107
ex82: -0.009
ex85: 0.3364
ex86: 0.0629


In [17]:
weight = coef_.x.copy()

In [18]:
weight[7] = weight[7] + 0.7

In [19]:
weight

array([ 0.9533741 , -1.66231918,  0.84880659,  4.521392  ,  2.01035548,
        0.77509809, -0.5466162 ,  0.19701345,  0.33705262,  2.79873232,
       -0.87630663,  2.28667754,  0.47155189,  6.49923451,  1.4390571 ])

In [20]:
weight_norm = weight / weight.sum()

In [21]:
weight_norm

array([ 0.04754247, -0.08289586,  0.04232794,  0.22547093,  0.10025159,
        0.03865228, -0.02725843,  0.00982459,  0.016808  ,  0.13956604,
       -0.0436993 ,  0.1140311 ,  0.02351516,  0.32410118,  0.07176231])

In [22]:
for i in range(len(weight_norm)):
    print("{}: {}".format(df.columns[i+1], round(weight_norm[i], 4)))

model275: 0.0475
model277: -0.0829
model277_2: 0.0423
model288_3: 0.2255
model294: 0.1003
model288_4: 0.0387
ex68: -0.0273
ex70: 0.0098
ex67: 0.0168
ex74: 0.1396
ex75: -0.0437
lgb014: 0.114
ex82: 0.0235
ex85: 0.3241
ex86: 0.0718
